In [193]:
import re
import glob
import json
import string
import gensim.models

In [194]:
from nltk.corpus import stopwords

In [195]:
def ouvrir_json(chemin):
    f = open(chemin, encoding="utf−8") 
    toto = json.load(f)
    f.close()
    return toto
def ecrire_json(dataset,chemin):
    donnee = json.dumps(dataset, indent =2,ensure_ascii=False) 
    w = open(chemin, "w",encoding = "utf-8")
    w.write(donnee)
    w.close()
    print("Jeu de données stocké dans %s"%chemin)

In [196]:
def preprocess(s):
	s_without_punct = s
	for char in string.punctuation:
		s_without_punct = s_without_punct.replace(char, ' ') # on replace les ponctuations par des espaces
	tokens = s_without_punct.split()
	return tokens

In [197]:
def get_corpus(corpus_path):
	# assume there's one sentence per line, tokens separated by whitespace
	# simple_preprocess: "Convert a document into a list of lowercase tokens, 
	# ignoring tokens that are too short or too long.""
	with open(corpus_path, mode='r', encoding='utf-8') as inFile:
		return [preprocess(line) for line in inFile.readlines()]

In [198]:
#les étapes du prétaitement du corpus 

# listes des tokens à gérer ou à supprimer
listeCaracteres={",":" ", "(":" ", ")":" ","\n":" ", "€":" ","=":" ", "+":" ",
                 "{":" ", "}":" ", "\\":" ", "\"":" ", "«": " ", "»":" ", "%":" ", ";":" ",
                 "/":" ", "_":" ", ": ":" ", " : ":" ",
                 "''":" "}
listeApostropheGerable={"d'":"d' ", "t'":"t' ", "j'":"j' ", "m'":"m' ", "s'":"s' ",
                        "l'":"l' ", "n'":"n' ", "qu'": "qu' ", "D'":"D' ", "T'":"T' ",
                        "J'":"J' ","M'":"M' ", "S'":"S' ", "L'":"L' ", "N'":"N' ", "Qu'": "Qu' "}
listePronomsInverses={"-je":" je", "-tu":" tu", "-il":" il", "-elle":" elle", "-on":" on", "-nous":" nous","-vous":" vous",
                      "-ils":" ils", "-elles":" elles", "-moi":" moi", "-ce":" ce"}
listePointsFinaux=[".", " .", " . ", "!", " !", " ! ",
                   "?", " ?", " ? "]
listeFourreTout={"-t":" t ", "-là":" là ", "@":" ", "$":""}
listeCar={"-":"", ":":""}

In [199]:
def cleanTextFromList(texte, liste):
    for punc, substitute in liste.items():
        texte=texte.replace(punc, substitute)
    return texte

In [200]:
def remplacementListe(listeCar, listeMots):
    newMots=[]
    for mot in listeMots:
        if mot[0] in listeCar.keys():
            substitute = listeCar[mot[0]]
            new=mot.replace(mot[0], substitute)
            newMots.append(new)
        if mot==mot.upper():
            mot = mot.lower()
            newMots.append(mot)
        else:
            newMots.append(mot)
    return(newMots)

In [201]:
#une fonction qui vise à gérer les mots capitalisés au début de la phrase
def gestionMajusculeDebutPhrase(liste):
    if len(liste)>0:
        PremierMot=liste[0]
        PremierMot=PremierMot.lower()
        liste[0]=PremierMot
        for point in listePointsFinaux:
            for i in range(len(liste)):
                if liste[i-1]==point:
                    motAModifier=liste[i]
                    motAModifier=motAModifier.lower()
                    liste[i]=motAModifier
    else:
        liste = liste
    return liste

def suppressionCaracteres(liste, listeCaractere):
    for caractere in listeCaractere:
        for mot in liste :
            if mot==caractere:
                liste.remove(mot)
            if mot=="":
                liste.remove(mot) 
    return liste
listeCar={"-":"", ":":""}

def remplacementListe(listeCar, listeMots):
    newMots=[]
    for mot in listeMots:
        if mot[0] in listeCar.keys():
            substitute = listeCar[mot[0]]
            new=mot.replace(mot[0], substitute)
            newMots.append(new)
        if mot==mot.upper():
            mot = mot.lower()
            newMots.append(mot)
        else:
            newMots.append(mot)
    return(newMots)

In [202]:
import re
def enlever_nombres(texte):
    texte = re.sub("[0-9\.]*", "", texte)
    return texte

In [203]:
def recupArticle_non_cap(texte):
    metaMots = [ ]
    texte=cleanTextFromList(texte, listeCaracteres)
    texte=cleanTextFromList(texte, listeApostropheGerable)
    texte=cleanTextFromList(texte, listePronomsInverses)
    texte=cleanTextFromList(texte, listeFourreTout)
    texte=enlever_nombres(texte)
    mots=texte.split()
    mots=gestionMajusculeDebutPhrase(mots)
    mots=suppressionCaracteres(mots, listePointsFinaux)
    mots=remplacementListe(listeCar, mots)
    for mot in mots:
        if mot.lower( ) not in stopwords .words("french"):
            metaMots.append(mot)
    return metaMots

In [204]:
#lemmatiseur qui ne transforme pas les mots en majuscul
def recupLemmatiseurGLAFF_1(cheminGLAFF):
    lemmatiseur={}
    with open(cheminGLAFF , "r", encoding="utf−8") as GLAFF:
        ligne=GLAFF.readline()
        while ligne:
            elems=re.split("\|", ligne)
            forme=elems[0]
            #forme=forme.capitalize()
            lemme=elems[2]
            #lemme=lemme.capitalize()
            freq=elems[6]
            lemmatiseur[forme]={"lemme":lemme , "freq":freq}
            ligne=GLAFF.readline()
    GLAFF.close()
    return lemmatiseur

In [205]:
def rechercheCorrespondances(mots, lemmatiseur):
    listeLemmes=[]
    for mot in mots:
        lemme=lemmatiseur.get(mot)
        if (lemme):
            listeLemmes.append(lemme["lemme"])
        else:
            listeLemmes.append(mot)
    return listeLemmes

In [206]:
def get_fre(corpus):
    dic_fre = { }
    for p in corpus:
        mots = recupArticle_non_cap(p)
        mots = rechercheCorrespondances(mots, lemmatiseurGLAFF)
        for m in mots:
            if m not in dic_fre:
                dic_fre[m] = 1
            else:
                dic_fre[m] +=1
    return dic_fre

In [207]:
#lemmatiseurGLAFF=recupLemmatiseurGLAFF_1("GLAFF-1.2.2/glaff-1.2.2.txt")

In [95]:
#ecrire_json(lemmatiseurGLAFF,"corpus/lem/lemGlaff.json")

Jeu de données stocké dans corpus/lem/lemGlaff.json


In [208]:
lemmatiseurGLAFF = ouvrir_json("corpus/lem/lemGlaff.json")

In [209]:
corpus_n = [ ]
for chemin in  glob.glob('corpus/appariements_faq\*.*')  :
    #print(chemin)
    corpus_q_r = ouvrir_json(chemin)
    for k,v in corpus_q_r.items():
        for l in v :
            if l != " ":
                corpus_n.append(l)

In [210]:
corpus_s = [ ]
for chemin in  glob.glob('corpus/corpus_TALN_2007-2013\*')[0:5]  : 
    f = open(chemin,"r",encoding="utf-8")
    corpus = f.read()
    corpus_s.append(corpus)

In [211]:
dic_fre_fac = get_fre(corpus_n)


In [212]:
dic_fre_sci = get_fre(corpus_s)

In [213]:
mots_fac_plus_fre = {k:v for k,v in sorted(dic_fre_fac.items (), key=lambda i:i[1], reverse=True )}


In [214]:
mots_sci_plus_fre = {k:v for k,v in sorted(dic_fre_sci.items (), key=lambda i:i[1], reverse=True )}

In [215]:
ecrire_json(mots_fac_plus_fre,"output/fre_fac.json")
ecrire_json(mots_sci_plus_fre,"output/fre_sci_5arti.json")

Jeu de données stocké dans output/fre_fac.json
Jeu de données stocké dans output/fre_sci_5arti.json


In [216]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2))

In [217]:
plus_fre_fac_300 = list(mots_fac_plus_fre.keys ())[0:301]
plus_fre_sci_300 = list(mots_sci_plus_fre.keys ())[0:301]
terme_candidat = intersection(plus_fre_fac_300, plus_fre_sci_300)

In [218]:
#print(terme_candidat) #??

['', 'trois', 'lors', 'proposer', 'tel', 'selon', 'ici', 'traiter', 'commander', 'tout', 'car', 'suivre', 'déjà', 'choisir', 'ainsi', 'très', 'utiliser', 'donc', "d'", 'non', 'se', 'encore', 'partir', 'commer', 'compter', 'aucun', 'alors', 'où', 'fois', 'lorsque', 'devoir', 'plusieurs', 'mettre', 'si', 'courir', 'nom', 'effectuer', 'cas', 'ne', 'chaque', 'possible', 'fin', 'réaliser', 'correspondre', 'le', 'faire', 'information', 'obtenir', 'bien', 'voir', 'prendre', 'première', 'généralement', 'afin', 'Si', 'autre', 'donner', 'deux', 'ce', 'permettre', 'après', '-', 'nombrer', 'avoir', 'rendre', 'régler', 'trouver', 'passer', 'rester', 'plaire', 'celer', 'que', 'être', 'avant', "c'est", 'pouvoir', 'concerner', 'également']


In [222]:
def get_GR(dic1,dic2):
    dic_GR = { }
    for m,fr in dic1.items():
        if m in list(dic2.keys()):
            GR =dic2[m]/fr
            dic_GR[m] = GR
    dic_GR_rank = {k:v for k,v in sorted(dic_GR.items (), key=lambda i:i[1], reverse=True )}
    return dic_GR_rank

In [225]:
GR = get_GR(dic_fre_fac,dic_fre_sci)
ecrire_json(GR,"output/GR.json")

Jeu de données stocké dans output/GR.json


In [236]:
def get_m_pre_dic (dic,m):
    n = 0
    dic_m = { }
    for k,v in dic.items():
        dic_m[k] =v
        n = n+1
        if n ==m:
            break
    return dic_m

def get_m_der_dic (dic,m):
    n = 0
    dic_m = { }
    for k,v in dic.items( ):
        n = n+1
        if n>len(dic)-m:
            dic_m[k]=v
            if n ==len(dic_m):
                break
    return dic_m


In [248]:
def get_infini_m(dic1,dic2,m):
    dic_inf = { }
    for mot,fr in dic1.items():
        if mot not in list(dic2.keys()):
            dic_inf[mot] =fr
    dic_inf_m = get_m_pre_dic (dic_inf,m)
    return dic_inf_m

In [249]:
GR_50_grand =  get_m_pre_dic (GR,50)
GR_50_petit =  get_m_der_dic (GR,50)
Infi_50_grand = get_infini_m(dic_fre_fac,dic_fre_sci,50)
ecrire_json(GR_50_grand,"output/GR_50_grand.json")
ecrire_json(GR_50_petit,"output/GR_50_petit.json")
ecrire_json(Infi_50_grand,"output/Infi_50_grand.json")

Jeu de données stocké dans output/GR_50_grand.json
Jeu de données stocké dans output/GR_50_petit.json
Jeu de données stocké dans output/Infi_50_grand.json


In [241]:
#print(GR_50_grand)
#print(GR_50_petit)